In [0]:
import os
import json
import nltk
import gensim
import numpy as np
import pandas as pd
from gensim import corpora, models, similarities
import pickle
import csv
import re
import tensorflow as tf

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
os.chdir("/content/drive/My Drive/Deep Learning")

In [0]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/drive/My Drive/EmbeddingDownloaded/glove.6B.50d.txt')
tmp_file = get_tmpfile("/content/drive/My Drive/EmbeddingDownloaded/word2vec.6B.50d.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [0]:
file=open('/content/drive/My Drive/Deep Learning/conversation.json')
data = json.load(file)

In [0]:
x = list()
y = list()

In [0]:
conversations = data['conversations']

In [0]:
for conversation in conversations:
  for position in range(len(conversation)):
    if position < (len(conversation)-1):
      x.append(conversation[position])
      y.append(conversation[position+1])

In [0]:
df = pd.DataFrame(list(zip(x, y)), columns = ['Question' , 'Answer'])

In [0]:
df['Answer'] = "startseq " + df['Answer'] + " endseq"

In [0]:
#Cleaning the Train Dataset
df['Question'] = df['Question'].apply(lambda x: x.lower())
df['Answer'] = df['Answer'].apply(lambda x: x.lower())


In [0]:
df['Question']  = df['Question'].str.replace("won't","will not")
df['Question']  = df['Question'].str.replace("ain't","am not")
df['Question']  = df['Question'].str.replace("'s"," is")
df['Question']  = df['Question'].str.replace("'m"," am")
df['Question']  = df['Question'].str.replace("'re'"," are")
df['Question']  = df['Question'].str.replace("can't","can not")
df['Question']  = df['Question'].str.replace("'ll"," will")
df['Question']  = df['Question'].str.replace("n't"," not")

df['Answer']  = df['Answer'].str.replace("won't","will not")
df['Answer']  = df['Answer'].str.replace("ain't","am not")
df['Answer']  = df['Answer'].str.replace("'s"," is")
df['Answer']  = df['Answer'].str.replace("'m"," am")
df['Answer']  = df['Answer'].str.replace("'re'"," are")
df['Answer']  = df['Answer'].str.replace("can't","can not")
df['Answer']  = df['Answer'].str.replace("'ll"," will")
df['Answer']  = df['Answer'].str.replace("n't"," not")

In [0]:
df['Question'] = df['Question'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
df['Question'] = df['Question'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['Answer'] = df['Answer'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
df['Answer'] = df['Answer'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces

In [0]:
df['Question'] = df['Question'].apply(lambda x: nltk.word_tokenize(x))
df['Answer'] = df['Answer'].apply(lambda x: nltk.word_tokenize(x))

In [0]:
encoder_input = []
decoder_input = []
decoder_output = []
for i in range(len(df['Question'])):
  temp_list = []
  for j in range(len(df['Answer'][i])):
    if j < (len(df['Answer'][i])-1):
      temp_list.append(df['Answer'][i][j])
      encoder_input.append(df['Answer'][i])
      decoder_input.append((temp_list).copy())
      decoder_output.append(df['Answer'][i][j+1])

In [0]:
#Encoder Sequence Length Maximum
encoder_sequence_length = max([len(x) for x in encoder_input])

In [0]:
#Decoder Sequence Length Maximum
decoder_sequence_length = max([len(x) for x in decoder_input])

In [0]:
def word_padding_post(words_list):
  temp_list = (words_list + ((decoder_sequence_length-len(words_list))  * ['zerovector']))
  return temp_list

def word_padding_pre(words_list):
  temp_list = (((encoder_sequence_length-len(words_list))  * ['zerovector'])+ words_list)
  return temp_list

In [0]:
def vector_conversion(word_list):
  return_list = []
  for word in word_list:
    if word in model.wv.vocab:
      return_list.append(model[word])
    else:
      return_list.append(np.zeros(50,dtype='float32'))
  return return_list

In [0]:
for Listno in range(len(encoder_input)):
  encoder_input[Listno] = word_padding_pre(encoder_input[Listno])
  encoder_input[Listno] = vector_conversion(encoder_input[Listno])
  decoder_input[Listno] = word_padding_pre(decoder_input[Listno])
  decoder_input[Listno] = vector_conversion(decoder_input[Listno])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [0]:
for index in range(len(decoder_output)):
  if decoder_output[index] in model.wv.vocab:
    decoder_output[index] = model[decoder_output[index]]
  else:
    decoder_output[index] = np.zeros(50,dtype='float32')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
model.similar_by_vector(np.array(encoder_input[0][0]))[0][0]

'kissane'

In [0]:
#Model Creation - Functional Type - Encoder
# Define an input sequence and process it.
encoder_Inputs = tf.keras.layers.Input(shape=(encoder_sequence_length,50))
encoder = tf.keras.layers.LSTM(256,return_state=True)
encoder_outputs, hidden_state, cell_state = encoder(encoder_Inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [hidden_state, cell_state]

In [0]:
#Model Creation - Functional Type - Decoder
decoder_Inputs = tf.keras.layers.Input(shape=(encoder_sequence_length,50))
decoder = tf.keras.layers.LSTM(256)
decoder_outputs = decoder(decoder_Inputs,initial_state = encoder_states)
decoder_final_output = tf.keras.layers.Dense(50, activation= 'softmax')(decoder_outputs)

In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
chatmodel = tf.keras.Model([encoder_Inputs, decoder_Inputs], decoder_final_output)
chatmodel.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [0]:
chatmodel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 30, 50)       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 30, 50)       0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 256), (None, 314368      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 256)          314368      input_6[0][0]                    
                                                                 lstm_4[0][1]                     
          

In [0]:
chatmodel.fit([encoder_input, decoder_input], np.array(decoder_output), batch_size=4, epochs=3, validation_split=0.2)

Train on 487 samples, validate on 122 samples
Epoch 1/3
487/487 [==============================] - 13s 27ms/sample - loss: -0.5674 - acc: 0.7823 - val_loss: -0.5574 - val_acc: 0.8033
Epoch 2/3
487/487 [==============================] - 13s 27ms/sample - loss: -0.5716 - acc: 0.7823 - val_loss: -0.5570 - val_acc: 0.8033
Epoch 3/3
487/487 [==============================] - 13s 27ms/sample - loss: -0.5738 - acc: 0.7823 - val_loss: -0.5565 - val_acc: 0.8033


In [0]:
model.similar_by_vector(chatmodel.predict([encoder_input[0:1], decoder_input[0:1]])[0])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('so', 0.8277916312217712),
 ('there', 0.819884181022644),
 ('can', 0.8198361396789551),
 ('very', 0.8169803023338318),
 ('be', 0.8159558176994324),
 ('rest', 0.8152173757553101),
 ('this', 0.8104703426361084),
 ('longer', 0.8100574612617493),
 ('all', 0.8090950846672058),
 ('it', 0.8074486255645752)]

In [0]:
chatmodel.predict([encoder_input[0:1], decoder_input[1:2]])

array([[4.18345667e-02, 1.39106335e-02, 1.36144314e-04, 1.72421991e-04,
        4.51465212e-02, 1.34517206e-02, 1.20810531e-04, 2.32319406e-04,
        1.23055637e-04, 1.63330473e-02, 2.80022301e-04, 4.21347171e-02,
        8.72102173e-05, 1.96870416e-04, 7.09684044e-02, 5.51160127e-02,
        3.05291936e-02, 1.23664979e-02, 2.97881837e-04, 6.80227167e-05,
        1.04624174e-04, 3.15407664e-02, 4.59026359e-02, 2.96458844e-02,
        5.37480079e-02, 5.32307786e-05, 6.37435369e-05, 2.10295245e-02,
        4.94366698e-02, 6.31455696e-05, 2.60017604e-01, 3.04463413e-02,
        1.08919252e-04, 8.02399809e-05, 2.99018662e-04, 2.13188861e-04,
        1.14575233e-02, 1.38959354e-02, 6.89863972e-03, 9.09245864e-05,
        1.92344945e-04, 7.75981368e-03, 7.21316028e-04, 4.12519015e-02,
        4.47543745e-04, 9.81524680e-03, 3.89741181e-04, 1.28284341e-03,
        4.77313471e-04, 3.90593037e-02]], dtype=float32)

In [0]:
model.similar_by_vector(decoder_output[1])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('am', 0.9999999403953552),
 ("'m", 0.7501711845397949),
 ('i', 0.7385112047195435),
 ('sorry', 0.7095773816108704),
 ('me', 0.7072381973266602),
 ('nobody', 0.7045166492462158),
 ('yes', 0.7002850770950317),
 ('tomorrow', 0.6995285153388977),
 ('happened', 0.6980783939361572),
 ('everybody', 0.694588303565979)]

In [0]:
df['Answer']

0                 [i, am, doing, well, how, about, you]
1                                   [i, am, also, good]
2                            [that, is, good, to, hear]
3                                         [yes, it, is]
4                                                  [hi]
5                                [how, are, you, doing]
6                                  [i, am, doing, well]
7                            [that, is, good, to, hear]
8                                         [yes, it, is]
9                   [can, i, help, you, with, anything]
10                          [yes, i, have, a, question]
11                           [what, is, your, question]
12                [could, i, borrow, a, cup, of, sugar]
13           [i, am, sorry, but, i, do, not, have, any]
14                                 [thank, you, anyway]
15                                        [no, problem]
16                [i, am, doing, well, how, about, you]
17                                  [i, am, also